In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, random_split, Subset
from torchvision import transforms
from PIL import Image
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import time
import os
import random

# ================================
# 1. Settings & Seed
# ================================
SEED = 42
torch.manual_seed(SEED)
np.random.seed(SEED)
random.seed(SEED)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

DEVICE = 'cuda' if torch.cuda.is_available() else 'mps' if torch.backends.mps.is_available() else 'cpu'
print(f"Using device: {DEVICE}")

CONFIG = {
    'img_dir': './cifar-10/train/train',
    'labels_csv': './cifar-10/trainLabels.csv',
    'test_dir': './cifar-10/test/test',
    'batch_size': 32,
    'learning_rate': 0.001,
    'epochs': 15,
    'val_split': 0.1,
    'data_subset_ratio': 0.2,
    'device': DEVICE
}

label_map = {
    'airplane': 0, 'automobile': 1, 'bird': 2, 'cat': 3, 'deer': 4,
    'dog': 5, 'frog': 6, 'horse': 7, 'ship': 8, 'truck': 9
}

# ================================
# 2. Dataset Class
# ================================
class Cifar10Dataset(Dataset):
    def __init__(self, csv_file=None, img_dir=None, transform=None, is_test=False):
        self.img_dir = img_dir
        self.transform = transform
        self.is_test = is_test

        if not is_test:
            self.data = pd.read_csv(csv_file)
            self.image_files = [f"{row[0]}.png" for row in self.data.values]
            self.labels = [label_map[row[1]] for row in self.data.values]
        else:
            self.image_files = sorted([f for f in os.listdir(img_dir) if f.endswith('.png')],
                                      key=lambda x: int(os.path.splitext(x)[0]))

    def __len__(self):
        return len(self.image_files)

    def __getitem__(self, idx):
        img_path = os.path.join(self.img_dir, self.image_files[idx])
        try:
            image = Image.open(img_path).convert('RGB')
        except:
            print(f"Error loading image {img_path}, using empty image instead")
            image = Image.new('RGB', (32, 32))
        
        if self.transform:
            image = self.transform(image)

        if self.is_test:
            label = -1
        else:
            label = self.labels[idx]

        return image, label

# ================================
# 3. Transforms
# ================================
transform = transforms.Compose([
    transforms.Resize((32, 32)),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465),
                         (0.2023, 0.1994, 0.2010))
])

# ================================
# 4. Model
# ================================
class BaseCNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 64, kernel_size=4)
        self.conv2 = nn.Conv2d(64, 32, kernel_size=4)
        self.pool = nn.MaxPool2d(2, 2)
        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(32 * 5 * 5, 256)
        self.fc2 = nn.Linear(256, 10)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.pool(self.relu(self.conv1(x)))
        x = self.pool(self.relu(self.conv2(x)))
        x = self.flatten(x)
        x = self.relu(self.fc1(x))
        return self.fc2(x)

# ================================
# 5. Training & Evaluation
# ================================
def get_dataloaders(subset_ratio=1.0, val_split=0.1):
    full_dataset = Cifar10Dataset(CONFIG['labels_csv'], CONFIG['img_dir'], transform)
    if subset_ratio < 1.0:
        subset_size = int(len(full_dataset) * subset_ratio)
        indices = torch.randperm(len(full_dataset))[:subset_size]
        dataset = Subset(full_dataset, indices)
    else:
        dataset = full_dataset

    val_size = int(len(dataset) * val_split)
    train_size = len(dataset) - val_size
    train_data, val_data = random_split(dataset, [train_size, val_size])

    num_workers = min(8, os.cpu_count() or 4)
    train_loader = DataLoader(train_data, batch_size=CONFIG['batch_size'], shuffle=True,
                              num_workers=num_workers, pin_memory=(DEVICE=='cuda'), persistent_workers=True)
    val_loader = DataLoader(val_data, batch_size=CONFIG['batch_size'], shuffle=False,
                            num_workers=max(1, num_workers//2), pin_memory=(DEVICE=='cuda'), persistent_workers=True)
    return train_loader, val_loader

def train_model(model, train_loader, val_loader):
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=CONFIG['learning_rate'])
    model.to(DEVICE)
    history = {'train_loss': [], 'val_loss': [], 'val_acc': []}

    for epoch in range(CONFIG['epochs']):
        model.train()
        train_loss = 0
        for images, labels in train_loader:
            images, labels = images.to(DEVICE), labels.to(DEVICE)
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            train_loss += loss.item()
        avg_train_loss = train_loss / len(train_loader)

        model.eval()
        val_loss, correct, total = 0, 0, 0
        with torch.no_grad():
            for images, labels in val_loader:
                images, labels = images.to(DEVICE), labels.to(DEVICE)
                outputs = model(images)
                val_loss += criterion(outputs, labels).item()
                _, predicted = torch.max(outputs, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()
        avg_val_loss = val_loss / len(val_loader)
        val_acc = 100 * correct / total

        history['train_loss'].append(avg_train_loss)
        history['val_loss'].append(avg_val_loss)
        history['val_acc'].append(val_acc)

        print(f"Epoch {epoch+1}/{CONFIG['epochs']} | Train Loss: {avg_train_loss:.4f} | "
              f"Val Loss: {avg_val_loss:.4f} | Val Acc: {val_acc:.2f}%")
    return history

# ================================
# 6. Test Evaluation
# ================================
def evaluate_test(model):
    test_dataset = Cifar10Dataset(img_dir=CONFIG['test_dir'], transform=transform, is_test=True)
    test_loader = DataLoader(test_dataset, batch_size=CONFIG['batch_size'], shuffle=False,
                             num_workers=4, pin_memory=(DEVICE=='cuda'))
    model.eval()
    preds = []
    with torch.no_grad():
        for images, _ in test_loader:
            images = images.to(DEVICE)
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            preds.extend(predicted.cpu().numpy())
    return preds

# ================================
# 7. Main Execution
# ================================
train_loader, val_loader = get_dataloaders(subset_ratio=CONFIG['data_subset_ratio'], val_split=CONFIG['val_split'])
model = BaseCNN()
start_time = time.time()
history = train_model(model, train_loader, val_loader)
duration = time.time() - start_time

print(f"\nTraining completed in {duration:.2f} seconds")

# Plot Loss
plt.figure(figsize=(10,5))
plt.plot(history['train_loss'], label='Train Loss')
plt.plot(history['val_loss'], label='Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Train vs Validation Loss')
plt.legend()
plt.grid(True)
plt.show()

# Evaluate test set
test_preds = evaluate_test(model)
print(f"Predictions for test set (first 10): {test_preds[:10]}")


Using device: cuda
